## 📊 FIFA 23 Player Stats - ML Pipeline Summary

### 🔍 Data Source
- Dataset downloaded from [KaggleHub](https://www.kaggle.com/datasets/bryanb/fifa-player-stats-database)
- Only **FIFA 23** data used to avoid duplicate player entries over years

### 🧹 Data Preprocessing
- Selected key features: Age, Nationality, Club, Contract Valid Until, Height, Weight, etc.
- Converted date columns to datetime
- Extracted `month` and `year` from `Joined`
- Cleaned numeric columns (Height, Weight, Release Clause) using regex
- Converted contract dates to year only

### 🔧 Feature Engineering
- Separated features into:
  - Categorical: `Nationality`, `Club`
  - Numerical: `Age`, `Height`, `Weight`, etc.
- Applied appropriate pipelines with `SimpleImputer`, `OneHotEncoder`, `StandardScaler`

### 🤖 Model Pipeline
- Used `XGBRegressor` wrapped in a `Pipeline` with preprocessing steps
- Trained using `train_test_split`
- Achieved a test score using `.score(X_test, y_test)`

### 💾 Model Saving
- Saved trained pipeline to `../models/pipeline.joblib` using `joblib`

### ✅ Notes
- One-hot encoding evaluated via `uniquer()` function to check cardinality
- Directory checks ensure smooth saving/loading


###### [*Summary created with the assistance of ChatGPT (OpenAI, 2025)*](https://chat.openai.com)



## Data Ingestion Pipeline

- Import important libraries
- Move data from cached kagglehub dataset folder into data/raw folder in project


In [82]:
import kagglehub
import shutil
import os
from pathlib import Path

## Dataset Selection

Although I'm loading datasets from **FIFA 17** to **FIFA 23**,  
I will only use **FIFA 23** for the actual analysis.  
This is because many players appear across multiple years,  
which could introduce bias or redundancy in the model.


In [83]:
path = kagglehub.dataset_download("bryanb/fifa-player-stats-database")


In [84]:
print("Path to dataset files:", path)

Path to dataset files: C:\Users\HP\.cache\kagglehub\datasets\bryanb\fifa-player-stats-database\versions\35


In [85]:
if Path("../data/raw/35"):
    print("data exists!")
else:    
    shutil.move(f"C:/Users/HP/.cache/kagglehub/datasets/bryanb/fifa-player-stats-database/versions/35","../data/raw")

data exists!


## Data Transformation Pipeline

- Transforming data (Clean dataset)

In [86]:
import numpy as np
import pandas as pd
import datetime
import re

In [87]:
df = pd.read_csv("../data/raw/35/FIFA23_official_data.csv")

In [88]:
df.columns

Index(['ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag', 'Overall',
       'Potential', 'Club', 'Club Logo', 'Value', 'Wage', 'Special',
       'Preferred Foot', 'International Reputation', 'Weak Foot',
       'Skill Moves', 'Work Rate', 'Body Type', 'Real Face', 'Position',
       'Joined', 'Loaned From', 'Contract Valid Until', 'Height', 'Weight',
       'Release Clause', 'Kit Number', 'Best Overall Rating'],
      dtype='object')

In [89]:
df

,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,Real Face,Position,Joined,Loaned From,Contract Valid Until,Height,Weight,Release Clause,Kit Number,Best Overall Rating
0,209658,L. Goretzka,27,https://cdn.sofifa.net/players/209/658/23_60.png,Germany,https://cdn.sofifa.net/flags/de.png,87,88,FC Bayern München,https://cdn.sofifa.net/teams/21/30.png,...,Yes,"<span class=""pos pos28"">SUB","Jul 1, 2018",NaN,2026,189cm,82kg,€157M,8.0,NaN
1,212198,Bruno Fernandes,27,https://cdn.sofifa.net/players/212/198/23_60.png,Portugal,https://cdn.sofifa.net/flags/pt.png,86,87,Manchester United,https://cdn.sofifa.net/teams/11/30.png,...,Yes,"<span class=""pos pos15"">LCM","Jan 30, 2020",NaN,2026,179cm,69kg,€155M,8.0,NaN
2,224334,M. Acuña,30,https://cdn.sofifa.net/players/224/334/23_60.png,Argentina,https://cdn.sofifa.net/flags/ar.png,85,85,Sevilla FC,https://cdn.sofifa.net/teams/481/30.png,...,No,"<span class=""pos pos7"">LB","Sep 14, 2020",NaN,2024,172cm,69kg,€97.7M,19.0,NaN
3,192985,K. De Bruyne,31,https://cdn.sofifa.net/players/192/985/23_60.png,Belgium,https://cdn.sofifa.net/flags/be.png,91,91,Manchester City,https://cdn.sofifa.net/teams/10/30.png,...,Yes,"<span class=""pos pos13"">RCM","Aug 30, 2015",NaN,2025,181cm,70kg,€198.9M,17.0,NaN
4,224232,N. Barella,25,https://cdn.sofifa.net/players/224/232/23_60.png,Italy,https://cdn.sofifa.net/flags/it.png,86,89,Inter,https://cdn.sofifa.net/teams/44/30.png,...,Yes,"<span class=""pos pos13"">RCM","Sep 1, 2020",NaN,2026,172cm,68kg,€154.4M,23.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17655,269526,Deng Xiongtao,19,https://cdn.sofifa.net/players/269/526/23_60.png,China PR,https://cdn.sofifa.net/flags/cn.png,48,61,Meizhou Hakka,https://cdn.sofifa.net/teams/114628/30.png,...,No,"<span class=""pos pos29"">RES","Apr 11, 2022",NaN,2027,190cm,78kg,€218K,35.0,NaN
17656,267946,22 Lim Jun Sub,17,https://cdn.sofifa.net/players/267/946/22_60.png,Korea Republic,https://cdn.sofifa.net/flags/kr.png,48,64,Jeju United FC,https://cdn.sofifa.net/teams/1478/30.png,...,No,"<span class=""pos pos29"">RES","Jan 1, 2022",NaN,2026,195cm,84kg,€188K,21.0,NaN
17657,270567,A. Demir,25,https://cdn.sofifa.net/players/270/567/23_60.png,Turkey,https://cdn.sofifa.net/flags/tr.png,51,56,Ümraniyespor,https://cdn.sofifa.net/teams/113796/30.png,...,No,"<span class=""pos pos29"">RES","Jun 6, 2021",NaN,2023,190cm,82kg,€142K,12.0,NaN
17658,256624,21 S. Czajor,18,https://cdn.sofifa.net/players/256/624/21_60.png,Poland,https://cdn.sofifa.net/flags/pl.png,50,65,Fleetwood Town,https://cdn.sofifa.net/teams/112260/30.png,...,No,"<span class=""pos pos29"">RES","Jan 1, 2020",NaN,2021,187cm,79kg,€214K,40.0,NaN


In [90]:
df_fifa = df[["Age","Nationality","Overall","Potential","Club","Joined","Contract Valid Until","Height","Weight","Release Clause"]].copy()

In [91]:
df_fifa

,Age,Nationality,Overall,Potential,Club,Joined,Contract Valid Until,Height,Weight,Release Clause
0,27,Germany,87,88,FC Bayern München,"Jul 1, 2018",2026,189cm,82kg,€157M
1,27,Portugal,86,87,Manchester United,"Jan 30, 2020",2026,179cm,69kg,€155M
2,30,Argentina,85,85,Sevilla FC,"Sep 14, 2020",2024,172cm,69kg,€97.7M
3,31,Belgium,91,91,Manchester City,"Aug 30, 2015",2025,181cm,70kg,€198.9M
4,25,Italy,86,89,Inter,"Sep 1, 2020",2026,172cm,68kg,€154.4M
...,...,...,...,...,...,...,...,...,...,...
17655,19,China PR,48,61,Meizhou Hakka,"Apr 11, 2022",2027,190cm,78kg,€218K
17656,17,Korea Republic,48,64,Jeju United FC,"Jan 1, 2022",2026,195cm,84kg,€188K
17657,25,Turkey,51,56,Ümraniyespor,"Jun 6, 2021",2023,190cm,82kg,€142K
17658,18,Poland,50,65,Fleetwood Town,"Jan 1, 2020",2021,187cm,79kg,€214K


In [92]:
df_fifa["Joined"] = pd.to_datetime(df_fifa.Joined)

In [93]:
df_fifa["month"] = df_fifa.Joined.dt.month
df_fifa["year"] = df_fifa.Joined.dt.year


In [94]:
df_fifa

,Age,Nationality,Overall,Potential,Club,Joined,Contract Valid Until,Height,Weight,Release Clause,month,year
0,27,Germany,87,88,FC Bayern München,2018-07-01,2026,189cm,82kg,€157M,7.0,2018.0
1,27,Portugal,86,87,Manchester United,2020-01-30,2026,179cm,69kg,€155M,1.0,2020.0
2,30,Argentina,85,85,Sevilla FC,2020-09-14,2024,172cm,69kg,€97.7M,9.0,2020.0
3,31,Belgium,91,91,Manchester City,2015-08-30,2025,181cm,70kg,€198.9M,8.0,2015.0
4,25,Italy,86,89,Inter,2020-09-01,2026,172cm,68kg,€154.4M,9.0,2020.0
...,...,...,...,...,...,...,...,...,...,...,...,...
17655,19,China PR,48,61,Meizhou Hakka,2022-04-11,2027,190cm,78kg,€218K,4.0,2022.0
17656,17,Korea Republic,48,64,Jeju United FC,2022-01-01,2026,195cm,84kg,€188K,1.0,2022.0
17657,25,Turkey,51,56,Ümraniyespor,2021-06-06,2023,190cm,82kg,€142K,6.0,2021.0
17658,18,Poland,50,65,Fleetwood Town,2020-01-01,2021,187cm,79kg,€214K,1.0,2020.0


In [95]:
df_fifa.isna().sum()

Age                        0
Nationality                0
Overall                    0
Potential                  0
Club                     211
Joined                  1098
Contract Valid Until     361
Height                     0
Weight                     0
Release Clause          1151
month                   1098
year                    1098
dtype: int64

In [96]:
def replace_char(x):
    return re.sub("[^0-9|.]","",x)

In [97]:
df_fifa.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17660 entries, 0 to 17659
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Age                   17660 non-null  int64         
 1   Nationality           17660 non-null  object        
 2   Overall               17660 non-null  int64         
 3   Potential             17660 non-null  int64         
 4   Club                  17449 non-null  object        
 5   Joined                16562 non-null  datetime64[ns]
 6   Contract Valid Until  17299 non-null  object        
 7   Height                17660 non-null  object        
 8   Weight                17660 non-null  object        
 9   Release Clause        16509 non-null  object        
 10  month                 16562 non-null  float64       
 11  year                  16562 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(3), object(6)
memory usage: 1.6+ M

In [98]:
df_fifa['Contract Valid Until'] = pd.to_datetime(df_fifa["Contract Valid Until"],format="mixed")

In [99]:
df_fifa['Contract Valid Until']  = df_fifa["Contract Valid Until"].dt.year

In [100]:
df_fifa['Contract Valid Until'].value_counts()

Contract Valid Until
2023.0    5077
2024.0    3670
2022.0    3437
2025.0    2133
2026.0    1117
2021.0     957
2027.0     399
2020.0     208
2019.0      52
2018.0      40
2017.0      36
2014.0      26
2015.0      25
2016.0      24
2013.0      18
2028.0      15
2009.0      15
2012.0      13
2011.0      11
2010.0      10
2007.0       8
2008.0       7
2031.0       1
Name: count, dtype: int64

In [101]:
df_fifa.Height = df_fifa.Height.map(replace_char)
df_fifa.Weight = df_fifa.Weight.map(replace_char)
df_fifa["Release Clause"] = df_fifa["Release Clause"].map(replace_char,na_action='ignore')

In [102]:
columns_list = ['Age', 'Nationality',  'Club', 'Joined',
       'Contract Valid Until', 'Height', 'Weight', 'Release Clause', 'month',
       'year','Potential','Overall']

In [103]:
df_fifa = df_fifa.loc[:,columns_list]

In [104]:
df['Name'].nunique()

17140

In [105]:
def uniquer(columns) -> None:
    for i in columns:
        print(f"Unique columns for {i} = {df_fifa[str(i)].nunique()}")


In [106]:
# Checking the number of unique values in selected columns
uniquer(['Nationality','Club','Contract Valid Until'])

Unique columns for Nationality = 161
Unique columns for Club = 926
Unique columns for Contract Valid Until = 23


### One Hot Encoding + Scikit-learn pipeline

In [107]:
from sklearn.pipeline import Pipeline,make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.ensemble import AdaBoostRegressor
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import joblib

xgbr = XGBRegressor()
scaler = StandardScaler()
encoder = OneHotEncoder(handle_unknown="ignore",sparse_output=False)
df_fifa_str = ["Nationality","Club"]
df_fifa_num = ["Age","Contract Valid Until","Height","Weight","Release Clause","month","year"]

In [108]:
str_pipeline = Pipeline(steps=[
    ("SimpleImputer",SimpleImputer(strategy="most_frequent")),
    ("OneHotEncoder",encoder)
    
])

In [109]:
num_pipeline = Pipeline(steps=[
    ("SimpleImputer",SimpleImputer(strategy="mean")),
    ("StandardScaler",scaler)
])

In [110]:
col_transformer = ColumnTransformer(transformers=[
    ("str_pipeline",str_pipeline,df_fifa_str),
    ("num_pipeline",num_pipeline,df_fifa_num)
],
remainder= "drop",
n_jobs=-1)

In [111]:
X = df_fifa.loc[:,['Age', 'Nationality', 'Club', 'Contract Valid Until',
       'Height', 'Weight', 'Release Clause', 'month', 'year']]
y = df_fifa.iloc[:,-1]

In [112]:
X.columns

Index(['Age', 'Nationality', 'Club', 'Contract Valid Until', 'Height',
       'Weight', 'Release Clause', 'month', 'year'],
      dtype='object')

In [113]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.3)

In [114]:
X_train.shape

(12362, 9)

In [115]:
X_train

,Age,Nationality,Club,Contract Valid Until,Height,Weight,Release Clause,month,year
12341,23,Bolivia,Club Atlético Nacional Potosí,2021.0,176,70,1.1,1.0,2019.0
10055,24,Latvia,Dundalk FC,2021.0,187,73,488,1.0,2021.0
2116,25,France,FC Girondins de Bordeaux,2024.0,186,81,7.1,1.0,2020.0
4264,28,England,FC Cincinnati,2022.0,188,87,906,8.0,2021.0
9001,23,Italy,SPAL,2024.0,182,77,1.9,7.0,2022.0
...,...,...,...,...,...,...,...,...,...
1108,22,Argentina,Club Atlético Banfield,2023.0,179,73,8.5,7.0,2018.0
13394,18,Romania,FC U Craiova 1948,2023.0,177,68,691,7.0,2022.0
4543,16,Turkey,Fenerbahçe SK,2025.0,176,61,6.7,7.0,2021.0
847,24,Spain,FC Zürich,2024.0,174,67,7.7,6.0,2021.0


In [116]:
# Create a pipeline for better readability and efficiency
pipefinal = make_pipeline(col_transformer, xgbr)

pipefinal.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(n_jobs=-1,
                                   transformers=[('str_pipeline',
                                                  Pipeline(steps=[('SimpleImputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('OneHotEncoder',
                                                                   OneHotEncoder(handle_unknown='ignore',
                                                                                 sparse_output=False))]),
                                                  ['Nationality', 'Club']),
                                                 ('num_pipeline',
                                                  Pipeline(steps=[('SimpleImputer',
                                                                   SimpleImputer()),
                                                                  ('StandardSc...
                              feature_types=None, feature_weights=None,
                              gamma=None, grow_policy=None,
                              importance_type=None,
                              interaction_constraints=None, learning_rate=None,
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints=None, multi_strategy=None,
                              n_estimators=None, n_jobs=None,
                              num_parallel_tree=None, ...))])

In [117]:
pipefinal.score(X_test,y_test)

0.9209163188934326

In [118]:
# Save the pipeline object
if not os.path.exists("../models"):
    os.makedirs("../models",exist_ok=True)
joblib.dump(pipefinal,"../models/pipeline.joblib")

['../models/pipeline.joblib']